This process is going to create a data frame that has cumulative over the season for that team. What that means is that the statistics are going to be the average up until that particular game. For example if we are focusing on the Atlanta Braves for the 2016 season each game is going to represent how that team is performing up until that game. If we are predicting the outcome of the 40th game we are using the average of all statistics for the first 39 games.

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns',500)
# used for sorting
from operator import itemgetter

In [3]:
df = pd.read_csv('data/pbp_data_mvp.csv')
df['Date'] = pd.to_datetime(df["Date"])

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (14,16,86,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# all team and years
team_list = df.home_team.unique()
year_list = df.Date.dt.year.unique()

In [9]:
df.head()

,Unnamed: 0,Date,game_in_series,day_of_week,away_team,away_league,away_team_game_number,home_team,home_league,home_team_game_number,away_team_score,home_team_score,number_of_outs,day_or_night,date_game_completed,forfeit_info,protest_info,park_id,attendance,time_of_game,away_line_scores,home_line_scores,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_catch_interference,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_pitch_balks,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_catch_interference,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_pitch_balks,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,hb_ump_id,hb_ump_name,1b_ump_id,1b_ump_name,2b_ump_id,2b_ump_name,3b_ump_id,3b_ump_name,lf_ump_id,lf_ump_name,rf_ump_id,rf_ump_name,away_team_manager_id,away_team_manager_name,home_team_manager_id,home_team_manager_name,win_pitch_id,win_pitch_name,lose_pitch_id,lose_pitch_name,save_pitch_id,save_pitch_name,game_win_rbi_batter_id,game_win_rbi_batter_name,away_start_pitch_id,away_start_pitch_name,home_start_pitch_id,home_start_pitch_name,away_player_1_id,away_player_1_name,away_player_1_def_pos,away_player_2_id,away_player_2_name,away_player_2_def_pos,away_player_3_id,away_player_3_name,away_player_3_def_pos,away_player_4_id,away_player_4_name,away_player_4_def_pos,away_player_5_id,away_player_5_name,away_player_5_def_pos,away_player_6_id,away_player_6_name,away_player_6_def_pos,away_player_7_id,away_player_7_name,away_player_7_def_pos,away_player_8_id,away_player_8_name,away_player_8_def_pos,away_player_9_id,away_player_9_name,away_player_9_def_pos,home_player_1_id,home_player_1_name,home_player_1_def_pos,home_player_2_id,home_player_2_name,home_player_2_def_pos,home_player_3_id,home_player_3_name,home_player_3_def_pos,home_player_4_id,home_player_4_name,home_player_4_def_pos,home_player_5_id,home_player_5_name,home_player_5_def_pos,home_player_6_id,home_player_6_name,home_player_6_def_pos,home_player_7_id,home_player_7_name,home_player_7_def_pos,home_player_8_id,home_player_8_name,home_player_8_def_pos,home_player_9_id,home_player_9_name,home_player_9_def_pos,additional_info,acquisition_info,year,outcome,home_OBP,away_OBP,home_AVG,away_AVG,home_singles,away_singles,home_SLG,away_SLG,home_BABIP,away_BABIP,home_ISO,away_ISO,home_PASO,away_PASO,home_total_bases,away_total_bases,home_runs_created,away_runs_created,home_wOBA,away_wOBA,game_number_of_season,home_won_last_game,away_won_last_game,id,target,away_outcome,home_outcome
0,0,2000-03-30,0,Thu,NYN,NL,2,CHN,NL,2,5,1,66,N,NaN,NaN,NaN,TOK01,55000.0,235,00001000004,00001000000,37,6,2,0,1,5,1,1,1,8,0,5,1,0,0,0,10,5,0,0,0,0,33,14,2,0,2,0,36,5,0,0,0,0,2,0,0,6,1,9,0,0,2,0,10,7,5,5,0,0,33,14,0,0,0,0,herna901,Angel Hernandez,fostm901,Marty Foster,kulpr901,Ron Kulpa,marsr901,Randy Marsh,NaN,(none),NaN,(none),valeb102,Bobby Valentine,bayld001,Don Baylor,cookd001,Dennis Cook,yound002,Danny Young,NaN,(none),agbab001,Benny Agbayani,reedr002,Rick Reed,farnk001,Kyle Farnsworth,hendr001,Rickey Henderson,7,hamid001,Darryl Hamilton,8,alfoe001,Edgardo Alfonzo,4,piazm001,Mike Piazza,2,ventr001,Robin Ventura,5,belld001,Derek Bell,9,zeilt001,Todd Zeile,3,ordor001,Rey Ordonez,6,reedr002,Rick Reed,1,youne001,Eric Young,4,bufod001,Damon Buford,8,sosas001,Sammy Sosa,9,gracm001,Mark Grace,3,rodrh001,Henry Rodriguez,7,andrs001,Shane Andrews,5,husoj001,Jeff Huson,6,giraj001,Joe Girardi,2,farnk001,Kyle Farnsworth,1,NaN,Y,2000,0,0.261905,0.319149,0.138889,0.162162,5,3,0.138889,0.297297,0.185185,0.156

These are rows that do not have any impact on the game in a way that can be useful for prediction. 

In [10]:
cols_to_drop = ['Unnamed: 0','forfeit_info','lf_ump_id','rf_ump_id','protest_info',
                    'date_game_completed','additional_info','save_pitch_id',
                    'game_win_rbi_batter_id','game_in_series','away_catch_interference',
                'home_catch_interference','away_pitch_balks','home_pitch_balks']
drop_cols_before_modeling = ['day_of_week','away_league',
                            'away_team_game_number','home_league',
                            'home_team_game_number','day_or_night','park_id','attendance',
                            'time_of_game','away_line_scores','home_line_scores','year','id',
                            'outcome']

df.drop(columns=cols_to_drop,inplace=True)
df.drop(columns=drop_cols_before_modeling,inplace=True)
df.drop(df.loc[:,'hb_ump_id':'acquisition_info'],axis=1,inplace=True)

Now I must get rid of all statistics that are not able to be averaged to create cumulative statistics over a year.

In [12]:
drop_for_cummean = ['number_of_outs','target','game_number_of_season',
                   'away_won_last_game','home_won_last_game','game_number_of_season']
df.drop(columns=drop_for_cummean,inplace=True)

In [13]:
# making dummy rows for use in putting games into year/team buckets
df['home_date'] = df.Date
df['away_date'] = df.Date

In [14]:
# initialize dict so that we can aggregate the stats of each team per year
# final dict is where we are going to be adding the aggregated stats
stat_dict = {}
final_dict = {}
for year in year_list:
    stat_dict[year] = {}
    final_dict[year] = {}
    for team in team_list:
        stat_dict[year][team] = []
        final_dict[year][team] = []

In [15]:
# this is putting games into the correct year/team combo
# we need both home and away games bc away games affect stats of the team
# this will end in lists that contain every game for every team for every year
# dict[year][team] = 161 lists of each games stats
for year in year_list:
    for team in team_list:
        # home game stats
        for game in df[(df.Date.dt.year == year)&(df.home_team == team)].filter(regex='home').values.tolist():
            stat_dict[year][team].append(np.array(game))
        # away game stats
        for game in df[(df.Date.dt.year == year)&(df.away_team == team)].filter(regex='away').values.tolist():
            stat_dict[year][team].append(np.array(game))
            

In [17]:
# have to sort each year/team array so that they are in the correct date order
for year in year_list:
    for team in team_list:
        stat_dict[year][team] = sorted(stat_dict[year][team],key=itemgetter(-1))

In [18]:
# putting date in the front to make it easier to work with
for year in year_list:
    for team in team_list:
        year_team_stats = []
        for game in stat_dict[year][team]:
            year_team_stats.append(np.insert(game[:-1],0,game[-1]))
        stat_dict[year][team] = year_team_stats

In [19]:
# this is aggregating the stats per year
# so each game is the mean of all stats of that game and all previous
# I am going to need to 
for year in year_list:
    for team in team_list:
        curr_game_number = 1
        aggregate_stats = np.array(stat_dict[year][team][0][2:])
        for game in stat_dict[year][team]:
            header_info = game[0:2]
            contents = game[2:]
            aggregate_stats_current_game = aggregate_stats/curr_game_number
            aggregate_stats = contents+aggregate_stats
            final_dict[year][team].append(np.concatenate((header_info,aggregate_stats_current_game)))
            curr_game_number +=1

In [20]:
agg_df = pd.DataFrame()

In [21]:
# creating the aggregate stats array
agg_array = []
for year in year_list:
    for team in team_list:
        for game in final_dict[year][team]:
            agg_array.append(game)

In [22]:
# making the array of all stats
agg_df = pd.DataFrame(agg_array)

In [45]:
away_data_column_names = list(df.filter(regex='away').columns)[1:-1] # gets rid of team name and date
home_data_column_names = list(df.filter(regex='home').columns)[1:-1] # gets rid of team name and date

In [24]:
game_basic_info_df = df.iloc[:,0:3]
for stat in away_data_column_names:
    game_basic_info_df[stat] = ''
for stat in home_data_column_names:
    game_basic_info_df[stat] = ''

In [25]:
final_matrix_for_df = []

In [26]:
for index,row in game_basic_info_df.iterrows():
    # append home team aggregate stats
    home_data = agg_df[(agg_df[0] == row.Date)&(agg_df[1]==row.home_team)].iloc[:,2:].to_numpy()
    # append away team aggregate stats
    away_data = agg_df[(agg_df[0] == row.Date)&(agg_df[1]==row.away_team)].iloc[:,2:].to_numpy()
    # create the full row
    full_row = np.concatenate((home_data[0],away_data[0]))
    temp_array = [row.Date,row.home_team,row.away_team]
    for individual_stat in full_row:
        temp_array.append(individual_stat)
    final_matrix_for_df.append(temp_array)

In [28]:
final_final_df = pd.DataFrame(final_matrix_for_df)

In [29]:
columns_for_output_df = ['Date','home_team','away_team']

In [30]:
for col_name in home_data_column_names:
    columns_for_output_df.append(col_name)
for col_name in away_data_column_names:
    columns_for_output_df.append(col_name)

In [31]:
final_final_df.columns = columns_for_output_df

In [1]:
# if this is your first time running this UNCOMMENT BELOW
#final_final_df.to_csv('data/aggregate_data.csv')